In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install torch torchvision timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import AutoFeatureExtractor, CvtForImageClassification
from PIL import Image
import requests
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data preprocessing
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Dataset and DataLoader setup
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/datasets/trainset', transform=train_transform)
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/datasets/testset', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model setup using CVT-13
feature_extractor = AutoFeatureExtractor.from_pretrained('microsoft/cvt-13', do_rescale=False)
model = CvtForImageClassification.from_pretrained('microsoft/cvt-13', num_labels=len(train_dataset.classes), ignore_mismatched_sizes=True).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training and evaluation
num_epochs = 10
train_losses = []
val_accuracies = []
val_f1_scores = []
val_precisions = []
val_recalls = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        inputs = feature_extractor(images=images, return_tensors="pt").to(device)
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs.logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    train_losses.append(train_loss / len(train_loader))

    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            inputs = feature_extractor(images=images, return_tensors="pt").to(device)
            outputs = model(**inputs)
            _, preds = torch.max(outputs.logits, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_accuracy = 100 * accuracy_score(all_labels, all_preds)
    f1 = 100 * f1_score(all_labels, all_preds, average='macro')
    precision = 100 * precision_score(all_labels, all_preds, average='macro')
    recall = 100 * recall_score(all_labels, all_preds, average='macro')

    val_accuracies.append(val_accuracy)
    val_f1_scores.append(f1)
    val_precisions.append(precision)
    val_recalls.append(recall)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Accuracy: {val_accuracy:.2f}%, '
          f'F1 Score: {f1:.2f}%, '
          f'Precision: {precision:.2f}%, '
          f'Recall: {recall:.2f}%')

# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/cvt_13_model.pth')

# Plotting
plt.figure(figsize=(14, 7))

# Plot Training Loss and Validation Accuracy
ax1 = plt.subplot(1, 2, 1)
lns1 = ax1.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', color='blue', linestyle='-')
ax2 = ax1.twinx()
lns2 = ax2.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='green', linestyle='--')

lns = lns1 + lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc='upper left')

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Training Loss', color='blue')
ax2.set_ylabel('Validation Accuracy', color='green')
ax1.tick_params(axis='y', labelcolor='blue')
ax2.tick_params(axis='y', labelcolor='green')

plt.title('Training Loss & Validation Accuracy')

# Plot Validation Metrics
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), val_f1_scores, label='F1 Score', linestyle='--', color='red')
plt.plot(range(1, num_epochs + 1), val_precisions, label='Precision', linestyle='-.', color='purple')
plt.plot(range(1, num_epochs + 1), val_recalls, label='Recall', linestyle=':', color='orange')
plt.title('Validation Metrics')
plt.xlabel('Epoch')
plt.ylabel('Percentage')
plt.legend(loc='best')

plt.tight_layout()
plt.show()


Some weights of CvtForImageClassification were not initialized from the model checkpoint at microsoft/cvt-13 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([10, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch [1/10], Train Loss: 1.2796, Train Accuracy: 56.00%, Val Accuracy: 70.40%, F1 Score: 66.79%, Precision: 79.07%, Recall: 70.40%
Epoch [2/10], Train Loss: 0.7807, Train Accuracy: 71.33%, Val Accuracy: 90.90%, F1 Score: 90.97%, Precision: 91.60%, Recall: 90.90%
Epoch [3/10], Train Loss: 0.6473, Train Accuracy: 75.83%, Val Accuracy: 93.60%, F1 Score: 93.61%, Precision: 93.80%, Recall: 93.60%
Epoch [4/10], Train Loss: 0.6043, Train Accuracy: 77.38%, Val Accuracy: 84.60%, F1 Score: 81.54%, Precision: 90.11%, Recall: 84.60%


In [ ]:
pip install libmr

  Preparing metadata (setup.py) ... done
  Created wheel for libmr: filename=libmr-0.1.9-cp310-cp310-linux_x86_64.whl size=549302 sha256=73698cba56c0198676d3199ea8dd61e38896db04e93324203fb3944b9bbb2ec2
  Stored in directory: /root/.cache/pip/wheels/8a/e0/45/ec5f4f802b034150d6f5735922408cc0278bf85582b2a1a954
Successfully built libmr


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import AutoFeatureExtractor, CvtForImageClassification, CvtConfig
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.special import softmax
import matplotlib.pyplot as plt
from PIL import Image

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Custom model class to access specific layer outputs
class CustomCvtForImageClassification(CvtForImageClassification):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, pixel_values, output_hidden_states=False):
        outputs = super().forward(pixel_values, output_hidden_states=True)
        if output_hidden_states:
            return outputs.hidden_states[-2]  # Adjust based on the model architecture
        return outputs.logits

# Initialize model
config = CvtConfig.from_pretrained('microsoft/cvt-13', num_labels=10)
model = CustomCvtForImageClassification(config)
model.load_state_dict(torch.load('/content/drive/MyDrive/mosquit_model7(CVT-13)/cvt_13_model.pth', map_location=device))
model.to(device)
model.eval()

# Data loaders
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/datasets/trainset', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/TestSet_Openmax', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32)

# Feature extractor setup, assuming no rescale is needed as images are already in [0,1]
feature_extractor = AutoFeatureExtractor.from_pretrained('microsoft/cvt-13', do_rescale=False)

# Get activations function
def get_activations(dataloader, model):
    model.eval()
    activations = []
    labels = []
    for images, labels_batch in dataloader:
        images = images.to(device)
        with torch.no_grad():
            activation = model(images, output_hidden_states=True)
        activations.append(activation.cpu().numpy())
        labels.extend(labels_batch.cpu().numpy())
    return np.array(activations), np.array(labels)

activations, labels = get_activations(train_loader, model)

# Dummy functions for MAVs and Weibull fitting (implement these based on your specifics)
def compute_mavs(activations, labels):
    return np.mean(activations, axis=0)

def fit_weibull(mavs, activations):
    return {}

mavs = compute_mavs(activations, labels)
weibull_models = fit_weibull(mavs, activations)

# OpenMax score application function (implement the real logic)
def apply_openmax(scores, mavs, weibull_models):
    return scores

# OpenMax inference
def openmax_inference(test_loader, model, feature_extractor, mavs, weibull_models, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []
    for images, labels_batch in test_loader:
        images = images.to(device)
        inputs = feature_extractor(images=images, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            scores = softmax(outputs.cpu().numpy(), axis=1)
            openmax_scores = apply_openmax(scores, mavs, weibull_models)
        pred = np.argmax(openmax_scores, axis=1)
        pred = np.where(np.max(openmax_scores, axis=1) < threshold, len(train_dataset.classes), pred)
        all_preds.extend(pred)
        all_labels.extend(labels_batch.numpy())
    return accuracy_score(all_labels, all_preds)

test_accuracy = openmax_inference(test_loader, model, feature_extractor, mavs, weibull_models)
print("Test accuracy with OpenMax:", test_accuracy)


/usr/local/lib/python3.10/dist-packages/transformers/models/convnext/feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


Test accuracy with OpenMax: 0.6972477064220184


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import AutoFeatureExtractor, CvtForImageClassification, CvtConfig
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.special import softmax
import matplotlib.pyplot as plt
from PIL import Image

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Custom model class to access specific layer outputs
class CustomCvtForImageClassification(CvtForImageClassification):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, pixel_values, output_hidden_states=False):
        outputs = super().forward(pixel_values, output_hidden_states=True)
        if output_hidden_states:
            return outputs.hidden_states[-2]  # Adjust based on the model architecture
        return outputs.logits

# Initialize model
config = CvtConfig.from_pretrained('microsoft/cvt-13', num_labels=10)
model = CustomCvtForImageClassification(config)
model.load_state_dict(torch.load('/content/drive/MyDrive/mosquit_model7(CVT-13)/cvt_13_model.pth', map_location=device))
model.to(device)
model.eval()

# Data loaders
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/datasets/trainset', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/TestSet_Openmax', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32)

# Feature extractor setup, assuming no rescale is needed as images are already in [0,1]
feature_extractor = AutoFeatureExtractor.from_pretrained('microsoft/cvt-13', do_rescale=False)

# Get activations function
def get_activations(dataloader, model):
    model.eval()
    activations = []
    labels = []
    for images, labels_batch in dataloader:
        images = images.to(device)
        with torch.no_grad():
            activation = model(images, output_hidden_states=True)
        activations.append(activation.cpu().numpy())
        labels.extend(labels_batch.cpu().numpy())
    return np.array(activations), np.array(labels)

activations, labels = get_activations(train_loader, model)

# Dummy functions for MAVs and Weibull fitting (implement these based on your specifics)
def compute_mavs(activations, labels):
    return np.mean(activations, axis=0)

def fit_weibull(mavs, activations):
    return {}

mavs = compute_mavs(activations, labels)
weibull_models = fit_weibull(mavs, activations)

# OpenMax score application function
def apply_openmax(scores, mavs, weibull_models):
    openmax_scores = []
    for score in scores:
        openmax_score = np.zeros(len(mavs) + 1)  # Add one for the unknown class
        # Reshape score to match the last dimensions of mavs
        score = np.expand_dims(score, axis=0)  # Add a new axis
        score = np.expand_dims(score, axis=0)  # Add two new axes
        score = np.expand_dims(score, axis=-1)  # Add one new axis
        distances = np.linalg.norm(mavs - score, axis=(1, 2, 3))
        for i in range(len(mavs)):
            w_score = weibull_models[i].w_score(distances[i])
            openmax_score[i] = score[i] * (1 - w_score)
        openmax_score[-1] = np.sum(score * [weibull_models[i].w_score(distances[i]) for i in range(len(mavs))])
        openmax_scores.append(openmax_score)
    return np.array(openmax_scores)



# OpenMax inference
def openmax_inference(test_loader, model, feature_extractor, mavs, weibull_models, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []
    for images, labels_batch in test_loader:
        images = images.to(device)
        inputs = feature_extractor(images=images, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            scores = softmax(outputs.cpu().numpy(), axis=1)
            openmax_scores = apply_openmax(scores, mavs, weibull_models)
        pred = np.argmax(openmax_scores, axis=1)
        pred = np.where(np.max(openmax_scores, axis=1) < threshold, len(train_dataset.classes), pred)
        all_preds.extend(pred)
        all_labels.extend(labels_batch.numpy())
    return accuracy_score(all_labels, all_preds)

test_accuracy = openmax_inference(test_loader, model, feature_extractor, mavs, weibull_models)
print("Test accuracy with OpenMax:", test_accuracy)
